# Recipes: time-based aggregation

The following use cases are covered here:
 - Use case 1: Sum daily sales, from individual item transactions.
 - Use case 2: Sum daily sales by store, from store/product sales.
 - Use case 3: Unify sensor measurements with the same timestamp.


## Use case 1: Aggregate daily sales

We have the sales log from a store, each item sale is represented by an event. They have a date-time, the price at which the product was sold, and the unit cost of the product.

We want to calculate total daily sales. So this is what we do:
- Create a uniform sampling with one tick per day (could be any other interval), at time `00:00:00`.
- Add up all sales that happened between `00:00:01` from the previous day, and the current tick at `00:00:00`.

In [ ]:
import pandas as pd
import temporian as tp

sales_data = pd.DataFrame(
    data=[
        ["2020-01-02 13:04", 3.0,  1.0],
        ["2020-01-02 13:04", 5.0,  2.0],  # duplicated timestamp
        ["2020-01-02 15:24", 7.0,  3.0],
        ["2020-01-03 13:45", 3.0,  1.0],
        ["2020-01-03 16:10", 7.0,  3.0],
        ["2020-01-03 17:30", 10.0, 5.0],
    ],
    columns=[
        "timestamp",
        "unit_price",
        "unit_cost",
    ],
)

sales_evset = tp.from_pandas(sales_data)
sales_evset

In [ ]:

# We want sales between these days
time_span = tp.event_set(timestamps=["2020-01-01 00:00", "2020-01-07 00:00"])

# Create ticks at 00:00
interval = tp.duration.days(1)
ticks = time_span.tick(interval)

ticks

In [ ]:

# Now we provide the ticks as sampling
moving_sum = sales_evset.moving_sum(window_length=interval, sampling=ticks)

moving_sum

In [ ]:

# Finally, rename and calculate profit
daily_sales = moving_sum.rename({"unit_price": "daily_revenue", "unit_cost": "daily_cost"})

daily_profit = (daily_sales["daily_revenue"] - daily_sales["daily_cost"]).rename("daily_profit")

daily_sales = tp.glue(daily_sales, daily_profit)

daily_sales


## Use case 2: Aggregate sales by store

In this case, let's assume that we've multiple stores with multiple products being sold.

We want to aggregate all product sales per store, so this is what we need to do:
- Ignore the `product_id` index.
- Unify daily sales per store.

In [ ]:
import pandas as pd
import temporian as tp


sales_data = pd.DataFrame(
    data=[
        # store 1
        ["2020-01-01", "store_1", "product_1", 300.0],
        ["2020-01-02", "store_1", "product_1", 450.0],
        ["2020-01-03", "store_1", "product_1", 600.0],
        ["2020-01-01", "store_1", "product_2", 100.0],
        ["2020-01-02", "store_1", "product_2", 250.0],
        ["2020-01-03", "store_1", "product_2", 100.0],
        # store 2
        ["2020-01-01", "store_2", "product_1", 900.0],
        ["2020-01-02", "store_2", "product_1", 750.0],
        ["2020-01-03", "store_2", "product_1", 750.0],
        ["2020-01-01", "store_2", "product_3", 20.0],
        ["2020-01-02", "store_2", "product_3", 40.0],
        ["2020-01-03", "store_2", "product_3", 30.0],
    ],
    columns=[
        "timestamp",
        "store_id",
        "product_id",
        "sales_usd",
    ],
)

# Load data indexed by store/product
sales_evset = tp.from_pandas(sales_data, indexes=["store_id", "product_id"])
sales_evset

Note that for each store/product we've the same 3 days of sales.

Now we want to calculate sales for each store, regardless of the `product_id`.

So, the first step is to remove that index.

In [ ]:
store_sales = sales_evset.drop_index("product_id")

store_sales

As you can see, now we've each timestamp duplicated, one for each product.

We want to discard the `product_id` information, and unify the timestamps adding up the sales.

In [ ]:
unique_days = store_sales.unique_timestamps()

store_daily_sales = store_sales["sales_usd"].moving_sum(window_length=tp.duration.days(1), sampling=unique_days)

store_daily_sales

## Use case 3: Unify sensor measurements

In this case we'll show how to avoid having events with duplicated timestamps.

It's different to the previous cases because:
 - The original timestamps must be preserved (i.e: can't use `tick()`).
 - The moving window can't cover two different timestamps (i.e: shortest `window_length` available)

Let's define some non-uniform timestamps to illustrate the use case.

In [ ]:
import temporian as tp

sensor_evset = tp.event_set(timestamps=[1.1, 2.01, 2.02, 2.02, 3.5, 3.51, 3.51, 4.5, 5.0],
                            features={"y": [1., 2., 3., 4., 5., 6., 7., 8., 9.],
                                      "z": [10., 20., 30., 40., 50., 60., 70., 80., 90.]
                                     }
                           )
sensor_evset

The first step is to create a new sampling removing the duplicated timestamps at `2.02` and `3.51`:

In [ ]:
# Remove duplicated timestamps
unique_t = sensor_evset.unique_timestamps()
unique_t

Next, we'll need a moving window but making sure that the window length doesn't overlap with two different timestamps at any point. It must be smaller than the smallest possible step.

In `tp.duration.shortest`, we've defined the shortest possible interval that can be represented with a `float64` timestamp at maximum resolution:

In [ ]:
shortest_length = tp.duration.shortest
shortest_length

Pretty small, right? Since null durations are not allowed, this is as close to zero as we can get. It's guaranteed that you'll never overlap two different timestamps using this.

Now we just need to run the aggregation function that we need, providing the `shortest_length` and the unique timestamps as sampling:

In [ ]:
unified_evset = sensor_evset.simple_moving_average(window_length=shortest_length, sampling=unique_t)
unified_evset

Of course, instead of the average value, other moving window operations like `moving_min` or `moving_max` could make more sense depending on the use case, this is just an illustrative example.

Also, keep in mind that this exact procedure would work well in an `EventSet` with multiple indexes, removing the duplicated timestamps in each index separately.

But let's keep the example simple for now 🙂